<h3><span style="color:#1fa2ad">CS F469: Information Retrieval</span></h3>
<h1><b>Text Processing</b></h1>
<h2><span style="color:#1fa2ad">Group 11</span></h2>

- Aditya Deshmukh
- Arshit Modi
- Guntaas Singh
- Saptarshi Das
- Siddarth Agrawal

*November 26, 2020*


# **PART 1**

Part 1 involves building an index over a subset of 89,095 documents from the Wikipedia corpus for processing free-text queries using the vector space model.

<h1 style="color:#1fa2ad"><b>Index Construction</b></h1>

- *Beautiful Soup* is used to extract the document text. Punctuation is removed and all terms are converted to lower-case. Other metadata, such as the document titles and IDs are also collected and stored.

- For fast retrieval, we use an **inverted index** implemented using Python dictionaries.

- The simple **in-memory sort-based indexing** algorithm is found to be sufficiently efficient for building the index in a reasonable amount of time.

- Additionally, we calculate and store normaization factors for all document vectors.

In [ ]:
# This cell contains various auxilary functions used throughout the notebook
# used during query processing

def getTermFreq(freq):
  '''
    int -> double

    Input : term_frequecny
    Output : normalized term frequency

    Description : 
    returns the log to the base 10 noramlized value 
  '''

  return 1 + math.log10(freq)

def getInvDocFreq(term, index):
  '''
    string,dict -> double

    Input : term,index
    Output : inverse document frequency

    Description : 
    returns the inverse document for the term
  '''

  num_docs = index['num_docs']
  doc_freq = len(index['postings'][term])
  return math.log10(num_docs / doc_freq)

def getTopKDocs(scores, k):
  '''
    list,int -> list

    Input : scores,k
    Output : list of documents

    Description : 
    returns a list of top 'k' documents with the highest score
  '''

  temp=list()

  for i in range(len(scores)):
      temp.append(i) 
  for n in range(0,k):
      maxi=n
      for i in range(n,len(temp)):
          if scores[temp[i]]>scores[temp[maxi]]:
              maxi=i
      temp1=temp[n]
      temp[n]=temp[maxi]
      temp[maxi]=temp1

  return temp[0:k]    


In [ ]:
def buildIndex(corpuspath):
  postings = {}
  num_docs = 0
  norm_factors = []
  doc_titles = []
  doc_ids = []

  i_doc = 0

  for foldername in os.listdir(corpuspath):
    
    # Constructing index from the 'AA' folder only
    if foldername != 'AA':
      continue
    
    for filename in os.listdir(os.path.join(corpuspath, foldername)):
      filepath = os.path.join(corpuspath, foldername, filename)

      # Opening the file in read only mode
      with open(filepath,'r',encoding='utf-8') as file:
        soup = file.read()
      soup = bsoup(soup)
      # Extracting all the documents
      docs = soup.find_all('doc')
      num_docs += len(docs)
      # Getting all the doc_titles and doc_ids
      doc_titles += [doc.get('title') for doc in docs]
      doc_ids += [doc.get('id') for doc in docs]


In [ ]:
for doc in docs:
        # Removing punctuations from the documents and tokenizing
        doctxt = doc.get_text()
        doctxt = doctxt.translate(str.maketrans('','',string.punctuation))
        tokens = word_tokenize(doctxt)
        tokens = [word.lower() for word in tokens]

        # Getting the term frequency for the term in the document
        doc_dict = {}
        for token in tokens:
          if token in doc_dict:
            doc_dict[token]=doc_dict[token]+1
          else:
            doc_dict[token]=1

In [ ]:
        # Calculating the L2 norm for the doc vector and adding 
        # the doc id and term frequency to the index
        norm_factors.append(0)
        for key in doc_dict:
          if key in postings:
            postings[key].append((i_doc,doc_dict[key]))
          else:
            postings[key] = [(i_doc, doc_dict[key])]
          norm_factors[i_doc] += getTermFreq(doc_dict[key]) ** 2

        norm_factors[i_doc] = math.sqrt(norm_factors[i_doc])
        i_doc = i_doc + 1

In [ ]:
  # Sorting the posting list for each term on the basis of document id
  for key in postings:
    postings[key] = sorted(postings[key])
  
  # Generate the index
  index = {
      'num_docs':num_docs,
      'postings':postings,
      'norm_factors':norm_factors,
      'doc_titles':doc_titles,
      'doc_ids':doc_ids
  }

  return index

In [ ]:
# Building the index
index = buildIndex('/content/drive/My Drive/IR assignment/corpus/Wikipedia')

Building Index for the 'AA' folder
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_33
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_46
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_52
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_79
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_59
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_10
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_20
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_07
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_80
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_68
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_67
	Processing /content/drive/My Drive/IR assignment/corpus/Wikipedia/AA/wiki_69
	Processing /content/drive/My

In [ ]:
# Pickling the index
pickle.dump(index, open('/content/drive/My Drive/IR assignment/index_AA.pkl', 'wb'))

In [ ]:
# Loads the generated index
index = pickle.load(open('/content/drive/My Drive/IR assignment/index_AA.pkl', 'rb'))

<h1 style="color:#1fa2ad"><b>Query Processing</b></h1>

- First, we convert the query to lowercase, remove punctuations and tokenize the query. 
- We calculate the cosine similarity score for the query vector (following *ltc* scheme) with every document vector (following *lnc* scheme).
- The top-K documents (K=10) are returned to the user.

In [ ]:
def processQuery(query, index, k):

  scores = [0] * index['num_docs']
  query_freq = {}

  # Tokenize the query after removing punctuations and converting to lowercase
  query = query.translate(str.maketrans('','',string.punctuation))
  tokens = word_tokenize(query)
  tokens = [word.lower() for word in tokens]

  # Find term frequency for the query
  for term in tokens:
    if term in query_freq:
      query_freq[term] += 1
    else:
      query_freq[term] = 1

In [ ]:
  # Calculating cosine similarity with certain optimizations:
    # Terms absent in the query do not contribute to the score, 
        # and are hence, ignored.
    # Document vector normalization factors computed duing index construction
        # are directly used.
  query_norm = 0
  for term in query_freq:
    if term not in index['postings']:
      continue
    queryWeight = getTermFreq(query_freq[term]) * getInvDocFreq(term, index)
    query_norm += queryWeight ** 2
    for i_doc, docFreq in index['postings'][term]:
      docWeight = getTermFreq(docFreq)
      scores[i_doc] += queryWeight * docWeight
  query_norm = math.sqrt(query_norm)
  
  for i_doc, norm_factor in enumerate(index['norm_factors']):
    scores[i_doc] /= (norm_factor * query_norm)

  return scores

In [ ]:
def testQuery(index, query):
    
  print(f'Retrieving documents for the query : {query} ...')

  # k to specify number of results to get
  k = 10
  query = query.lower()
  start = time()
  scores = processQuery(query, index, k)
  res = getTopKDocs(scores, k)
  end = time()
  print(f"Query processing took {end-start:.3f} sec.")

  # Printing the results
  print('Printing retrieved documents ... ')
  for rank, i_doc in enumerate(res):
    doc_freq = -1
    print(f"{rank+1:3}: [DocID : {i_doc:5}]  " + 
          "DocTitle : {index['doc_titles'][i_doc]:35}  " + 
          "Score : {scores[i_doc]:.3f})"
         )


In [ ]:
# Evaluating the query
query = input("Please enter the query to be searched : ") # animal cell
testQuery(index, query)

Please enter the query to be searched : animal cell
Retrieving documents for the query : animal cell ...
Query processing took 0.104 sec.
Printing retrieved documents ... 
  1: [DocID : 61241] DocTitle : K cell                              (Score : 0.388)
  2: [DocID : 80157] DocTitle : Cell potential                      (Score : 0.388)
  3: [DocID : 58583] DocTitle : Animal (disambiguation)             (Score : 0.212)
  4: [DocID : 35211] DocTitle : Cell Cycle (journal)                (Score : 0.210)
  5: [DocID : 19815] DocTitle : Amphibian (disambiguation)          (Score : 0.199)
  6: [DocID : 20277] DocTitle : Flora (disambiguation)              (Score : 0.160)
  7: [DocID : 34066] DocTitle : Eight-cell stage                    (Score : 0.147)
  8: [DocID :  9588] DocTitle : Plasmolysis                         (Score : 0.145)
  9: [DocID : 86094] DocTitle : Mannan                              (Score : 0.141)
 10: [DocID : 72645] DocTitle : Zygote                              (Sco

# **PART 2**

We now focus on improving the performance of our model by using heuristics like title-weighing and query expansion

<h1 style="color:#1fa2ad"><b>Improvement : Title Weighing</b></h1>


- The title information supplied in the documents can be very useful while retrieving the documents as it is a solid indicator of information need presented by the user in terms of query. 
- We take into account by calculating the **jaccard similarity** of document title and query and merging it with the cosine score using a suitable parameter.
- By the method of trial and error, we ran our experiment for various values of lambda, and found that **lambda = 0.5** works well for our model. 


In [ ]:
def jaccard(title,query):

  # Tokenizing the document title and query
  l1=regexp_tokenize(title.lower(), "[\w']+")
  l2=regexp_tokenize(query.lower(), "[\w']+")

  s1=set()
  s2=set()

  # Count for storing the intersection of title and query terms
  count=0
  for i in l1:
    s1.add(i)
    s2.add(i)
  
  for i in l2:
    if i in s1:
        s1.remove(i)
        count=count+1
    s2.add(i)
  
  return count/len(s2)

In [ ]:
def processQuery(query, index, k, lambda1):

  # Tokenize the query after removing punctuations
  query = query.translate(str.maketrans('','',string.punctuation))
  scores = [0] * index['num_docs']
  query_freq = {}
  tokens = word_tokenize(query)
  tokens = [word.lower() for word in tokens]
  
  # Calculate frequency for query terms
  for term in tokens:
    if term in query_freq:
      query_freq[term] += 1
    else:
      query_freq[term] = 1

In [ ]:
  # Calculating cosine similarity and adding an extra lamda * jaccard 
  # similarity between document title and query-
  query_norm = 0
  for term in query_freq:
    if term not in index['postings']:
      continue
    queryWeight = getTermFreq(query_freq[term]) * getInvDocFreq(term, index)
    query_norm += queryWeight ** 2
    for i_doc, docFreq in index['postings'][term]:
      docWeight = getTermFreq(docFreq)
      scores[i_doc] += queryWeight * docWeight
  query_norm = math.sqrt(query_norm)

  for i_doc, norm_factor in enumerate(index['norm_factors']):
    scores[i_doc] /= (norm_factor * query_norm)
    scores[i_doc]=scores[i_doc]+jaccard(index['doc_titles'][i_doc],query)*lambda1
  
  return scores

In [ ]:
# Lamda_title_weigh is the factor by which we want to weigh in the relevance 
# between document title and query
lambda_title_weigh = 0.5

In [ ]:
# Evaluating the query
query = input("Please enter the query to be searched : ")
testQuery(index ,query , lambda_title_weigh)

In [ ]:
# Evaluating the query
query = input("Please enter the query to be searched : ")
testQuery(index ,query , lambda_title_weigh)

Please enter the query to be searched : Teenage mutant ninja Turtles
Retrieving documents for the query : Teenage mutant ninja Turtles using title weighing ...
Query processing took 0.786 sec.
Printing retrieved documents ... 
  1: [DocID : 88893] DocTitle : Teenage Mutant Ninja Turtles        (Score : 0.599)
  2: [DocID : 21221] DocTitle : Ninja                               (Score : 0.186)
  3: [DocID : 19779] DocTitle : Belgian hip hop                     (Score : 0.145)
  4: [DocID :  4593] DocTitle : Ninja Tune                          (Score : 0.128)
  5: [DocID :  6019] DocTitle : High Falls, New York                (Score : 0.115)
  6: [DocID : 39720] DocTitle : Atari Teenage Riot                  (Score : 0.107)
  7: [DocID : 47398] DocTitle : Sean Astin                          (Score : 0.083)
  8: [DocID : 26063] DocTitle : Mae Whitman                         (Score : 0.079)
  9: [DocID : 43634] DocTitle : Corey Feldman                       (Score : 0.077)
 10: [DocID : 214

<h1 style="color:#1fa2ad"><b>Improvement : Automatic Query Expansion</b></h1>

- **Word mismatch**: Users of IR systems often use different words to describe a concept in their queries than those used in the documents. Our IR system ignores such terms, which can negatively affect the ranking performance.
- Since the meaning of a word can be heavily context-dependent, we cannot depend on the synonyms as much as the original query terms. 
- To this end, we assign a weight (discount_rate)(adjustable parameter, less than 1) to each synonym added to the query, using which the terms are weighed when calculating the cosine similarity.
- By the method of trial and error, we ran our experiment for various values of discount_rate and number of synonyms (expansion_factor), and found that **discount_rate = 0.7** and **expansion_factor = 5** works well for our model.

In [ ]:
def expandTokenList(token_list, mask, index, expansion_factor=5, 
                    always_expand=True, discount_rate=0.7):

  exp_token_list = token_list.copy()

  for org_word in token_list:
    syn_count = 0

    # if the word is found in the index do not expand query corresponding to the 
    # term and check also check whether query expansion is true or not
    if org_word in index['postings'] and not always_expand:
      continue

    # Get lemma for the query term (equal to synset name)
    for synset in wordnet.synsets(org_word):
      lemma = synset.lemmas()[0] 

      # add synonym to the new list if the not already present in the list,
      # synonym should be present in the index and number of synonyms extracted
      # should be less than expansion factor
      if (syn_count < expansion_factor) and 
            (lemma.name().lower() not in exp_token_list) and 
            (lemma.name().lower() in index['postings']):
        exp_token_list.append(lemma.name().lower())
        mask.append(discount_rate)
        syn_count += 1

  return exp_token_list, mask

In [ ]:
def processQuery(query, index, k, queryExpansion=False):
  
  # Tokenize the query after removing punctuations
  query = query.translate(str.maketrans('','',string.punctuation))
  scores = [0] * index['num_docs']
  query_freq = {}
  mask_dict = {}
  tokens = word_tokenize(query)
  tokens = [word.lower() for word in tokens]
  mask = [1] * len(tokens)

  # Call query expansion
  if queryExpansion:
    tokens, mask = expandTokenList(
        tokens, mask, index, expansion_factor=5, 
        always_expand=True, discount_rate=0.7
    )
    print(f"Expanded query: {str(tokens)}")

  # Compute frequency of query terms
  for i in range(len(tokens)):
    term = tokens[i]
    m_val = mask[i]

    if term in query_freq:
      query_freq[term] += 1
      mask_dict[term] = max(mask_dict[term], m_val)
    else:
      query_freq[term] = 1
      mask_dict[term] = m_val

In [ ]:
  # Calculating cosine similarity and also introduced a mask factor which (generally)
  # reduces the weight of the synonyms taken into consideration
  query_norm = 0
  for term in query_freq:

    if term not in index['postings']:
      print(f"DEBUG: Corpus missing term '{term}'")
      continue

    queryWeight = getTermFreq(query_freq[term]) * getInvDocFreq(term, index)
    query_norm += queryWeight ** 2
    for i_doc, docFreq in index['postings'][term]:
      docWeight = getTermFreq(docFreq)
      scores[i_doc] += queryWeight * docWeight * mask_dict[term]
  query_norm = math.sqrt(query_norm)
  
  for i_doc, norm_factor in enumerate(index['norm_factors']):
    scores[i_doc] /= (norm_factor * query_norm)
  
  return scores

In [ ]:
# Evaluating the query
query = input("Please enter the query to be searched : ")
testQuery(index ,query ,queryExpansion=True)

Please enter the query to be searched : competition
Retrieving documents for the query : competition using query expansion ...
Expanded query: ['competition', 'contest', 'rival']
Query processing took 0.110 sec.
Printing retrieved documents ... 
  1: [DocID : 63439] DocTitle : Imperfect competition               (Score : 0.149)
  2: [DocID : 32350] DocTitle : Snowboarding at the 2002 Winter Olympics (Score : 0.114)
  3: [DocID : 42993] DocTitle : Interactive Fiction Competition     (Score : 0.105)
  4: [DocID : 50968] DocTitle : Gandalf (theorem prover)            (Score : 0.090)
  5: [DocID : 32346] DocTitle : Luge at the 2002 Winter Olympics    (Score : 0.088)
  6: [DocID : 84232] DocTitle : International Obfuscated C Code Contest (Score : 0.082)
  7: [DocID : 15654] DocTitle : Eurovision Song Contest 1956        (Score : 0.078)
  8: [DocID : 86759] DocTitle : Miss World                          (Score : 0.077)
  9: [DocID : 32352] DocTitle : Bobsleigh at the 2002 Winter Olympics (Sc

<h1 style="color:#1fa2ad"><b>Thank you</b></h1>
<h2 style="color:#1fa2ad"><b>Contributions</b></h2>

| Task | Aditya Deshmukh | Arshit Modi | Guntaas Singh | Saptarshi Das | Siddarth Agrawal |
| :-: | :-: | :-: | :-: | :-: | :-: |
| Index Construction  | X   |  |  | X   |  |
| Query Processing    |  |  | X   | | |
| Query Evaluation    | | X   |  |  | X   |
| I1: Title Weighing  | | X |  |  | X   |
| I2: Query Expansion | X   |  | X   |  | |
